In [1]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 

from flask import Flask, request
from transformers import (LlamaForCausalLM, CodeLlamaTokenizer,
                          BitsAndBytesConfig)
import torch

from data_management.lm_example import LmExample 
from tactic_gen.train_codellama import (collate_input, CONF_NAME, load_config,
                                        get_tokenizer)


/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_LOC = "/home/ubuntu/coq-modeling/models/codellama-freq-eval"
CHECKPOINT_NUM = 360 

model_path = os.path.join(MODEL_LOC, f"checkpoint-{CHECKPOINT_NUM}")
model_conf = load_config(os.path.join(MODEL_LOC, CONF_NAME))

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
# model = LlamaForCausalLM.from_pretrained(
#     model_path, quantization_config=quantization_config
# )

model_name = "codellama/CodeLlama-7b-hf"
model = LlamaForCausalLM.from_pretrained(
    model_name, quantization_config=quantization_config
)

tokenizer = get_tokenizer(model_conf) 
device = "cuda" 

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.48s/it]


In [3]:
test_in = "Theorem andb_true_elim2 : forall b c : bool, andb b c = true -> c = true.\n  intros b c.\n  destruct b as [|].\n  -  destruct c as [|].\n    +  reflexivity.\n    +  simpl.  intro.  rewrite -> H.  reflexivity.\n  -  destruct c as [|].\n    +  simpl.  intro.  reflexivity.\n    +  simpl.  intro.  rewrite -> H.  reflexivity.<THM-SEP>\n\ntrue && true = true -> true = true<GOAL-SEP>\n\ntrue && false = true -> false = true"
test_out = "\n    + "

In [4]:
#collated_in = collate_input(test_in)
prompt = '''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
'''

In [5]:
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")

In [6]:
output = model.generate(input_ids, max_new_tokens=200)
output = output[0].to("cpu")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/ubuntu/coq-modeling/venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [7]:
only_output = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)

In [12]:
2 in output

False

In [10]:
tokenizer.eos_token_id

2

In [9]:
print(only_output)


Theorem andb_true_elim3 : forall b c : bool, andb b c = true -> b = true.
  intros b c.
  destruct b as [|].
  -  destruct c as [|].
    +  reflexivity.
    +  simpl.  intro.  rewrite -> H.  reflexivity.
  -  destruct c as [|].
    +  simpl.  intro.  reflexivity.
    +  simpl.  intro.  rewrite -> H.  reflexivity.<THM-SEP>

true && true = true -> true = true<GOAL-SEP>

true && false = true -> false = true
<TACTIC>

Theorem andb_true_elim4 : forall b c : bool, andb b c = true -> b = true.
  intros b c.
 
